In [73]:
import os
import hashlib
import marshal
from pyeoskit import eosapi, wallet, db
from pyeoskit import config

db.reset()
config.main_token = 'UUOS'

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')

wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')

def publish_contract(account_name, code, abi):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        eosapi.set_contract(account_name, code, abi, 1)
    return True

#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])

def run_test_code(code, account_name='test11111111'):
    publish_contract(account_name, code, abi)
    try:
        r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'active'})
        print(r['processed']['action_traces'][0]['console'])
    except Exception as e:
        print(e)


https://github.com/EOSIO/eosio.contracts/tree/master/contracts/eosio.token

```C++
void token::transfer( name    from,
                      name    to,
                      asset   quantity,
                      string  memo )
{
    check( from != to, "cannot transfer to self" );
    require_auth( from );
    check( is_account( to ), "to account does not exist");
    auto sym = quantity.symbol.code();
    stats statstable( _self, sym.raw() );
    const auto& st = statstable.get( sym.raw() );

    require_recipient( from );
    require_recipient( to );

    check( quantity.is_valid(), "invalid quantity" );
    check( quantity.amount > 0, "must transfer positive quantity" );
    check( quantity.symbol == st.supply.symbol, "symbol precision mismatch" );
    check( memo.size() <= 256, "memo has more than 256 bytes" );

    auto payer = has_auth( to ) ? to : from;

    sub_balance( from, quantity );
    add_balance( to, quantity, payer );
}
```

In [ ]:
key1 = 'EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV'
key2 = 'EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV'
eosapi.create_account('test11111111', 'test11111112', key1, key2, 64*1024, 1.0, 1.0)

In [ ]:
help(eosapi.create_account)

In [ ]:
help(eosapi.create_account)

### require_recipient
def require_recipient(account: int) -> None:

In [75]:
name = 'test11111111'
code = '''
import struct
def apply(receiver, code, action):
    print(n2s(receiver), n2s(code), n2s(action))
    data = read_action_data()
    from_, to, amount, symbol = struct.unpack('QQQ8s', data[:32])
    print(n2s(from_), n2s(to), symbol)
'''
abi = ''
publish_contract(name, code, abi)

True

In [142]:
name = 'test11111112'
code = '''
import struct
eosio_token = N('eosio.token')
transfer = N('transfer')
def apply(receiver, code, action):
    if code == eosio_token and transfer == action:
        data = read_action_data()
        from_, to, amount, symbol = struct.unpack('QQq8s', data[:32])
        if symbol==b'\\x04UUOS\\x00\\x00\\x00' and to == N('test11111112'):
            print('Do your homework here!')
        return
'''
abi = ''
publish_contract(name, code, abi)


True

In [143]:
r = eosapi.transfer('test11111112', 'test11111111', 0.1)
print(r['processed']['action_traces'][0]['inline_traces'][0]['console'])
print(r['processed']['action_traces'][0]['inline_traces'][1]['console'])
print(r['processed']['elapsed'])


b' B\x08!\x84\x90\xb1\xca\x10B\x08!\x84\x90\xb1\xca\xe8\x03\x00\x00\x00\x00\x00\x00\x04UUOS\x00\x00\x00\x00'
done!

1667
